In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/ms4610project"

Mounted at /content/drive
/content/drive/MyDrive/ms4610project


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sun Dec 12 12:59:21 2021
@author: nikhi
"""

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from sklearn import metrics


In [3]:
df = pd.read_csv("Training Data_2021.csv")
df = pd.get_dummies(df, columns=['mvar47'],drop_first = True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:

removed = ['mvar22','mvar26', 'mvar15', 'mvar35', 'mvar30', 'mvar23', 
           'mvar45', 'mvar11', 'mvar41', 'mvar31', 'mvar40']

In [5]:
categorical = ['mvar47_L']

df = df.drop(removed, axis = 1)

df_cat = df[categorical]
df_num = df.drop(categorical,axis = 1)

df_num = df_num.apply(pd.to_numeric, errors='coerce')
df_cat = df_cat.apply(pd.to_numeric, errors='coerce')


In [6]:

df_num = df_num.fillna(df_num.mean())
for column in df_cat.columns:
    df_cat[column].fillna(df_cat[column].mode()[0], inplace=True)


In [7]:
df_cat.nunique()
df_cat = pd.get_dummies(df_cat,columns = categorical,drop_first = True)

In [8]:


y = df['default_ind']

df_num = df_num.drop(['application_key','default_ind'],axis = 1)

standardizer = StandardScaler()
df_stand = standardizer.fit_transform(df_num)
df_stand = pd.DataFrame(df_stand)
df_stand.columns = df_num.columns

In [9]:
"""
pca = PCA(n_components=len(df_stand.columns))
principalComponents = pca.fit_transform(df_stand)


# Plot the explained variances
features = range(pca.n_components_)
plt.bar(features, pca.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(features)


comp = 5

pc = pd.DataFrame(data = principalComponents[:,:comp]) 

"""

"\npca = PCA(n_components=len(df_stand.columns))\nprincipalComponents = pca.fit_transform(df_stand)\n\n\n# Plot the explained variances\nfeatures = range(pca.n_components_)\nplt.bar(features, pca.explained_variance_ratio_, color='black')\nplt.xlabel('PCA features')\nplt.ylabel('variance %')\nplt.xticks(features)\n\n\ncomp = 5\n\npc = pd.DataFrame(data = principalComponents[:,:comp]) \n\n"

In [13]:
findata = pd.merge(df_stand, df_cat, left_index=True, right_index=True)

# models = [#LogisticRegression(),GaussianNB(),
#           #SGDClassifier(loss = 'modified_huber',class_weight = "balanced",shuffle = True,random_state = 101)
#           #KNeighborsClassifier(n_neighbors = 10),
#           RandomForestClassifier(n_estimators=100,n_jobs = -1,random_state = 101,min_samples_leaf = 30),
#           #SVC(kernel = 'linear',C=0.025,random_state = 101)
        
        
#         ]

In [12]:
from sklearn.metrics import accuracy_score, log_loss , classification_report, roc_auc_score


In [15]:
x_train,x_val,y_train,y_val = train_test_split(findata,y,test_size=0.1)


def evaluate_model(model):
  model.fit(x_train, y_train)
  y_pred_train = model.predict(x_train)
  y_pred_val = model.predict(x_val)
  print(classification_report(y_val, y_pred_val))

  y_pred_train = model.predict_proba(x_train)
  y_pred_val = model.predict_proba(x_val)
  print(f"training log loss {log_loss(y_train,y_pred_train)} test log loss {log_loss(y_val, y_pred_val)}")
  print(f" roc auc score :{roc_auc_score(y_val, y_pred_val[:,1])}")
  return model

In [17]:
for n_estimators in [50,100,200,500,1000]:
    #model.fit(X_train,y_train)
    model = RandomForestClassifier(n_estimators=n_estimators,n_jobs = -1,random_state = 101,min_samples_leaf = 30)
    print(f"================{n_estimators}=====================\n")
    evaluate_model(model)

================50=====================

              precision    recall  f1-score   support

           0       0.80      0.93      0.86      5968
           1       0.68      0.39      0.50      2332

    accuracy                           0.78      8300
   macro avg       0.74      0.66      0.68      8300
weighted avg       0.76      0.78      0.76      8300

training log loss 0.43444077747350424 test log loss 0.4835621308107755
 roc auc score :0.7843662124239512
================100=====================

              precision    recall  f1-score   support

           0       0.80      0.93      0.86      5968
           1       0.68      0.39      0.50      2332

    accuracy                           0.78      8300
   macro avg       0.74      0.66      0.68      8300
weighted avg       0.76      0.78      0.76      8300

training log loss 0.43384934289570176 test log loss 0.48280308158305824
 roc auc score :0.7852737829314951
================200=====================

        

# 500 is good 

In [18]:
model = RandomForestClassifier(n_estimators=500,n_jobs = -1,random_state = 101,min_samples_leaf = 30)
model.fit(findata,y)

RandomForestClassifier(min_samples_leaf=30, n_estimators=500, n_jobs=-1,
                       random_state=101)

In [19]:
df1 = pd.read_csv("Test Data_2021.csv")
df1 = pd.get_dummies(df1, columns=['mvar47'],drop_first = True)

df1 = df1.drop(removed, axis = 1)

df1_cat = df1[categorical]
df1_num = df1.drop(categorical,axis = 1)

df1_num = df1_num.apply(pd.to_numeric, errors='coerce')
df1_cat = df1_cat.apply(pd.to_numeric, errors='coerce')

df1_num = df1_num.fillna(df1_num.mean())
for column in df1_cat.columns:
    df1_cat[column].fillna(df1_cat[column].mode()[0], inplace=True)

df1_cat.nunique()
df1_cat = pd.get_dummies(df1_cat,columns = categorical,drop_first = True)


df1_num = df1_num.drop(['application_key'],axis = 1)

standardizer1 = StandardScaler()
df1_stand = standardizer1.fit_transform(df1_num)
df1_stand = pd.DataFrame(df1_stand)
df1_stand.columns = df1_num.columns

findata1 = pd.merge(df1_stand, df1_cat, left_index=True, right_index=True)


y1_pred = np.around(model.predict(findata1))

submission = df1[['application_key']].copy()
submission['default_key'] = y1_pred

submission.to_csv("Group_29_6.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
